In [1]:
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import sklearn.gaussian_process.kernels as kl

from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import RBFInterpolator
from scipy.interpolate import NearestNDInterpolator
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA


import models
import em

2022-05-17 14:29:13.772957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 14:29:13.772985: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Transformer 0.65um 

<img src="../img/transf.png" alt="drawing" width="200"/> 


Octagonal transformer in TSMC 65 um. 1_1T, 1_2T, 2_1T

Before running this notebook make sure to [prepare](data_prepare_tmtt_transf.ipynb) the data.


In [2]:
#model definition
class PreProcess():
    def __init__(self, model, whitening = False, standardize = False) -> None:
        self.model = model
        self.x_scaler = StandardScaler() if standardize else None
        self.pca = PCA(whiten=True) if whitening else None
    
    def fit(self, x_train, y_train):
        if self.pca:
            self.x_mean = np.mean(x_train, axis=0)
            x_train = x_train - self.x_mean
            x_train = self.pca.fit_transform(x_train)
        elif self.x_scaler:
            x_train = self.x_scaler.fit_transform(x_train)

        self.model.fit(x_train, y_train)

    def predict(self, x):
        if self.pca:
            x = x - self.x_mean
            x = self.pca.transform(x)
        elif self.x_scaler:
            x = self.x_scaler.transform(x)
        
        return self.model.predict(x)

class RBFInterpolatorModel:
    def __init__(self, degree=1) -> None:
        self.degree = degree
        
    def fit(self, x_train, y_train):
        self.model = RBFInterpolator(x_train, y_train, 
                     degree=self.degree, smoothing=0.00000001, neighbors=4000)

    def predict(self, x):
        return self.model(x)

class GaussianProcessRegressorModel:
    def __init__(self, kernel) -> None:
        self.model = GaussianProcessRegressor(kernel=kernel, alpha=1e-8)
        
    def fit(self, x_train, y_train):
        self.model.fit(x_train, y_train)

    def predict(self, x):
        return self.model.predict(x)


class NearestNDInterpolatorModel:
    def fit(self, x_train, y_train):
        self.model = NearestNDInterpolator(x_train, y_train)

    def predict(self, x_test):
        return self.model(x_test)

class ANNModel:
    def __init__(self, layers = None, degree = 5) -> None:
       
        self.poly     = PolynomialFeatures(degree)
        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        self.layers = layers if layers else [256, 512]

    def fit(self, x_train, y_train):
        x_t = self.poly.fit_transform(x_train)  
        x_t = self.x_scaler.fit_transform(x_t)

        y_t = self.y_scaler.fit_transform(y_train)

        activation = 'relu'
        output='linear'
        epochs = 1000
        loss="mse" 
        optimizer = keras.optimizers.Adam(learning_rate=0.0001)


        inputs = keras.Input(shape=(x_t.shape[1],), name='parameters')
        
        lay = inputs

        for n in self.layers:
            lay = keras.layers.Dense(n, activation=activation, 
               kernel_regularizer=keras.regularizers.L2(0.000001), 
               activity_regularizer=keras.regularizers.L2(0.001))(lay)

        outputs = keras.layers.Dense(y_t.shape[1], activation=output, 
            kernel_regularizer=keras.regularizers.L2(0.000001))(lay)
        
        self.model = keras.Model(inputs=inputs, outputs=outputs)
        self.model.compile(
            loss=loss,
            optimizer=optimizer)
        
        self.history = self.model.fit(x_t, y_t, 
                    epochs = epochs, 
                    batch_size= 64, 
                    verbose = 0)


    def predict(self, x_test):
        return self.y_scaler.inverse_transform(
            self.model.predict(self.x_scaler.transform(
                self.poly.transform(x_test))))

In [3]:

def model_atf():
    _list = []
    _list.append(PreProcess(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)), standardize=True))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=7), whitening=True))

    _list.append(PreProcess(RandomForestRegressor(max_depth= 15, max_samples=0.8), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [256, 512]))

    
    return _list

def model_at():
    _list = []
    _list.append(PreProcess(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)), standardize=True))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=6), whitening=True))
    
    _list.append(PreProcess(RandomForestRegressor(), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [512, 1024]))
    
    return _list

def model_list():
    _list = []

    _list.append(PreProcess(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)), standardize=True))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=4), whitening=True))

    _list.append(PreProcess(RandomForestRegressor(), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [256, 512]))

    
    return _list


def srf_models():
    _list = []
    _list.append(PreProcess(RBFInterpolatorModel(degree=2), whitening=True))
    _list.append(PreProcess(RBFInterpolatorModel(degree=3), whitening=True))
    return _list



sub_spaces = [(1,1, ""), (1,2, ""),(2,1, "")]

def ranges(df_train, df_test):
    df_test = df_test[df_test.Dinp > 20]
    df_test = df_test[df_test.Wp > 6]
    df_test = df_test[df_test.Wp < 14]
    df_test = df_test[df_test.Dinp < 180 ]
    df_test = df_test[df_test.Dins > 20]
    df_test = df_test[df_test.Ws > 6]
    df_test = df_test[df_test.Ws < 14]
    df_test = df_test[df_test.Dins < 180 ]

    return  df_train, df_test

def ranges_28(df_train, df_test):
    df_test = df_test[df_test.Dinp > 20]
    df_test = df_test[df_test.Wp > 6]
    df_test = df_test[df_test.Wp < 14]
    df_test = df_test[df_test.Dinp < 180 ]
    df_test = df_test[df_test.Dins > 20]
    df_test = df_test[df_test.Ws > 6]
    df_test = df_test[df_test.Ws < 14]
    df_test = df_test[df_test.Dins < 180 ]

    df_test = df_test[df_test.freq < 28.5 ]
    df_test = df_test[df_test.freq > 27.5 ]

    return  df_train, df_test


mdls_atf = model_atf()
mdls_at = model_at()

In [29]:
# AllTF
np.random.seed(1234)
tf.random.set_seed(1234)
errors_all_TF  = [[((None, None, None, None, None),(None, None, None, None, None),(None, None, None, None, None)) for _ in range(6)]]

freq, x_train, y_train, x_test, y_test, srf_data = models.load_data("../data/transf_65nm/", filter= ranges_28)

print(x_train.shape, x_test.shape)
print(x_train[0], x_test[0])

(704304, 7) (65, 7)
[ 41.001   2.      1.    130.      9.     95.      9.   ] [ 28.001   1.      1.    130.      7.     67.      8.   ]


In [34]:
#GPR AllTF
try:
    mdls_atf[0].fit(x_train, y_train)
    pred = mdls_atf[0].predict(x_test)
    errors_all_TF[0][0] = em.mape_lq_diff(freq, pred, y_test)
except MemoryError as error:
    print(error, file= sys.stderr)

Unable to allocate 3.61 TiB for an array with shape (704304, 704304) and data type float64


In [35]:
#KR AllTF
try:
    mdls_atf[1].fit(x_train, y_train)
    pred = mdls_atf[1].predict(x_test)
    errors_all_TF[0][1] = em.mape_lq_diff(freq, pred, y_test)
except MemoryError as error:
    print(error, file= sys.stderr)

Unable to allocate 3.61 TiB for an array with shape (704304, 704304) and data type float64


In [36]:
#RBF AllTF 
# - degree 7 w 4000 neigbours over 12Hrs no results
# moved to degree 6 and 2000 neigbours - Prediction resutls might differ, but are bad. 
# For the sake of argument is the same as this strategy is not recomended anyway.
try:
    mdls_atf[2].fit(x_train, y_train)
    pred = mdls_atf[2].predict(x_test)
    errors_all_TF[0][2] = em.mape_lq_diff(freq, pred, y_test)
except MemoryError as error:
    print(error, file= sys.stderr)

In [37]:
#RFR AllTF
try:
    mdls_atf[3].fit(x_train, y_train)
    pred = mdls_atf[3].predict(x_test)
    errors_all_TF[0][3] = em.mape_lq_diff(freq, pred, y_test)
except MemoryError as error:
    print(error, file= sys.stderr)

In [38]:
#NND AllTF
mdls_atf[4].fit(x_train, y_train)
pred = mdls_atf[4].predict(x_test)
errors_all_TF[0][4] = em.mape_lq_diff(freq, pred, y_test)

In [39]:
#ANN AllTF
mdls_atf[5].fit(x_train, y_train)
pred = mdls_atf[5].predict(x_test)
errors_all_TF[0][5] = em.mape_lq_diff(freq, pred, y_test)

2022-04-23 16:37:16.297885: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2231235072 exceeds 10% of free system memory.


In [40]:
print("AllTF")
models.error_to_csv(errors_all_TF)

AllTF
Lp, None, None, None, None, 325.92, 86.75, 81.53, 8.33, 121.71, 42.49, 145.84, 5.82, 
Qp, None, None, None, None, 218.66, 103.63, 87.76, 17.81, 4734.25, 418.82, 184.71, 26.73, 
Ls, None, None, None, None, 10671.42, 303.86, 1815.32, 37.16, 6088.05, 139.35, 808.26, 22.17, 
Qs, None, None, None, None, 2263.12, 165.57, 2528.58, 58.84, 25199.16, 759.65, 925.00, 39.02, 
k, None, None, None, None, 1689.02, 235.61, 81.98, 9.88, 171.10, 53.16, 290.53, 14.69, 


In [41]:
# AllT
np.random.seed(1234)
tf.random.set_seed(1234)
errors_all_T = [[((None, None, None, None, None),(None, None, None, None, None),(None, None, None, None, None)) for _ in range(6)]]

freq, x_train, y_train, x_test, y_test, _ = models.load_data("../data/transf_65nm/", 28,filter=ranges)
print(x_train.shape)
print(x_train[0])

#errors_all_T.append(models.test_it(mdls_at, freq, x_train, y_train, x_test, y_test))


(3504, 6)
[  2   1 189   8 121   5]


In [69]:
#GPR  AllT
mdls_at[0].fit(x_train, y_train)
pred = mdls_at[0].predict(x_test)
errors_all_T[0][0] = em.mape_lq_diff(freq, pred, y_test)

/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better

In [17]:
#KR  AllT
mdls_at[1].fit(x_train, y_train)
pred = mdls_at[1].predict(x_test)
errors_all_T[0][1] = em.mape_lq_diff(freq, pred, y_test)

NameError: name 'kl' is not defined

In [64]:
#RBF  AllT
mdls_at[2].fit(x_train, y_train)
pred = mdls_at[2].predict(x_test)
errors_all_T[0][2] = em.mape_lq_diff(freq, pred, y_test)


In [65]:
#RFR  AllT
mdls_at[3].fit(x_train, y_train)
pred = mdls_at[3].predict(x_test)
errors_all_T[0][3] = em.mape_lq_diff(freq, pred, y_test)

In [66]:
#NND  AllT
mdls_at[4].fit(x_train, y_train)
pred = mdls_at[4].predict(x_test)
errors_all_T[0][4] = em.mape_lq_diff(freq, pred, y_test)

In [67]:
#ANN  AllT
mdls_at[5].fit(x_train, y_train)
pred = mdls_at[5].predict(x_test)
errors_all_T[0][5] = em.mape_lq_diff(freq, pred, y_test)

In [42]:

print("AllT")
models.error_to_csv(errors_all_T)

AllT
Lp, None, None, None, None, None, None, None, None, None, None, None, None, 
Qp, None, None, None, None, None, None, None, None, None, None, None, None, 
Ls, None, None, None, None, None, None, None, None, None, None, None, None, 
Qs, None, None, None, None, None, None, None, None, None, None, None, None, 
k, None, None, None, None, None, None, None, None, None, None, None, None, 
Lp, 15.90, 0.46, 18.91, 2.41, 8.37, 0.34, 27.17, 4.49, 121.71, 42.49, 12.67, 2.51, 
Qp, 17.50, 1.74, 200.71, 15.63, 8.69, 1.32, 58.15, 10.79, 4734.25, 418.82, 72.97, 10.99, 
Ls, 40.45, 1.31, 248.25, 6.40, 16.34, 0.72, 742.15, 18.02, 6088.05, 139.35, 597.58, 12.69, 
Qs, 42.54, 2.35, 190.05, 15.43, 16.92, 1.64, 774.93, 25.39, 25199.16, 759.65, 566.16, 16.33, 
k, 29.79, 0.84, 47.90, 3.22, 8.52, 0.41, 167.96, 8.35, 171.10, 53.16, 52.83, 2.91, 


In [27]:
# NO SRF
np.random.seed(1234)
tf.random.set_seed(1234)
errors = []

for subsp in sub_spaces:
    print(f"Subspace: {subsp}")
    freq, x_train, y_train, x_test, y_test, _ = models.load_data("../data/transf_65nm/", 28, nt=subsp, n_samples=1000, filter=ranges)
    errors.append(models.test_it(model_list(), freq, x_train, y_train, x_test, y_test))

print("No SRF filter")
models.error_to_csv(errors,line_headers = [f'Np={nt[0]} Ns={nt[1]}' for nt in sub_spaces])

Subspace: (1, 1, '')
Subspace: (1, 2, '')
Subspace: (2, 1, '')
No SRF filter
Np=1 Ns=1, Lp, 0.34, 0.13, 1.04, 0.22, 0.35, 0.15, 9.10, 2.28, 10.62, 3.61, 0.88, 0.35, 
Np=1 Ns=1, Qp, 6.99, 1.75, 4.86, 1.81, 3.31, 1.43, 14.47, 8.58, 21.78, 4.33, 18.11, 3.46, 
Np=1 Ns=1, Ls, 0.23, 0.08, 1.03, 0.31, 0.26, 0.08, 15.97, 2.92, 10.15, 4.22, 1.59, 0.57, 
Np=1 Ns=1, Qs, 2.68, 0.75, 12.32, 4.04, 2.40, 0.80, 20.50, 8.29, 12.80, 4.70, 12.63, 3.37, 
Np=1 Ns=1, k, 0.46, 0.22, 0.81, 0.25, 0.41, 0.16, 10.00, 3.39, 25.38, 9.02, 4.64, 1.11, 
Np=1 Ns=2, Lp, 13.73, 0.89, 30.06, 1.72, 9.07, 0.62, 32.79, 3.27, 151.58, 11.40, 11.24, 0.98, 
Np=1 Ns=2, Qp, 15.66, 2.82, 31.44, 3.99, 10.81, 2.11, 235.22, 23.24, 167.15, 20.39, 81.36, 7.33, 
Np=1 Ns=2, Ls, 27.26, 1.55, 57.35, 3.11, 17.92, 1.12, 69.67, 8.75, 258.33, 20.09, 31.75, 2.37, 
Np=1 Ns=2, Qs, 28.21, 2.97, 57.39, 5.00, 19.06, 2.76, 314.74, 27.08, 355.21, 27.44, 109.77, 9.82, 
Np=1 Ns=2, k, 6.23, 0.59, 17.56, 1.28, 3.93, 0.44, 15.34, 5.68, 280.66, 24.72, 5.03,

In [25]:
# SRF Filter at + 10GHz
np.random.seed(1234)
tf.random.set_seed(1234)
errors = []

for subsp in sub_spaces:
    print(f"Subspace: {subsp}")
    freq, x_train, y_train, x_test, y_test, _ = models.load_data("../data/transf_65nm/", 28, srf=38, nt=subsp, n_samples=1000, filter=ranges)
    errors.append(models.test_it(model_list(), freq, x_train, y_train, x_test, y_test))


print("SRF filter")
models.error_to_csv(errors,line_headers = [f'Np={nt[0]} Ns={nt[1]}' for nt in sub_spaces])


Subspace: (1, 1, '')
Subspace: (1, 2, '')
Subspace: (2, 1, '')
SRF filter
Np=1 Ns=1, Lp, 0.34, 0.13, 1.04, 0.22, 0.35, 0.15, 9.10, 2.28, 10.62, 3.61, 0.88, 0.35, 
Np=1 Ns=1, Qp, 6.99, 1.75, 4.86, 1.81, 3.31, 1.43, 14.47, 8.58, 21.78, 4.33, 18.11, 3.46, 
Np=1 Ns=1, Ls, 0.23, 0.08, 1.03, 0.31, 0.26, 0.08, 15.97, 2.92, 10.15, 4.22, 1.59, 0.57, 
Np=1 Ns=1, Qs, 2.68, 0.75, 12.32, 4.04, 2.40, 0.80, 20.50, 8.29, 12.80, 4.70, 12.63, 3.37, 
Np=1 Ns=1, k, 0.46, 0.22, 0.81, 0.25, 0.41, 0.16, 10.00, 3.39, 25.38, 9.02, 4.64, 1.11, 
Np=1 Ns=2, Lp, 0.62, 0.17, 0.52, 0.16, 0.65, 0.18, 5.16, 1.98, 6.22, 2.54, 2.00, 0.60, 
Np=1 Ns=2, Qp, 5.22, 1.91, 6.04, 2.37, 4.96, 1.86, 22.09, 11.81, 27.33, 9.10, 6.44, 2.99, 
Np=1 Ns=2, Ls, 0.47, 0.15, 0.44, 0.24, 0.45, 0.14, 12.69, 5.53, 16.20, 5.92, 1.97, 0.72, 
Np=1 Ns=2, Qs, 3.00, 0.94, 4.03, 1.83, 4.40, 1.27, 17.95, 11.38, 15.69, 6.09, 14.22, 3.62, 
Np=1 Ns=2, k, 0.69, 0.26, 1.16, 0.47, 0.50, 0.16, 11.93, 5.62, 29.01, 12.19, 5.19, 1.50, 
Np=2 Ns=1, Lp, 0.14, 0.0

In [19]:
# SRF Filter at + 10GHz
np.random.seed(1234)
tf.random.set_seed(1234)
srf_errors = []


for subsp in sub_spaces:
    print(f"Subspace: {subsp}")
    freq, x_train, y_train, x_test, y_test, srf_data = models.load_data("../data/transf_65nm/", 28, srf=38, nt=subsp, n_samples=1000, filter=ranges)    
    srf_errors.append(models.test_it_srf(srf_models(), srf_data))

print("SRF filter error")
models.error_to_csv(srf_errors, line_headers = [f'Np={nt[0]} Ns={nt[1]}' for nt in sub_spaces], error_headers = ['SRFp, ', 'SRFs, '])

Subspace: (1, 1, '')
Subspace: (1, 2, '')
Subspace: (2, 1, '')
SRF filter error
Np=1 Ns=1, SRFp, 1.72, 0.32, 1.50, 0.32, 
Np=1 Ns=1, SRFs, 6.00, 1.83, 5.99, 1.81, 
Np=1 Ns=2, SRFp, 58.93, 15.54, 58.90, 15.53, 
Np=1 Ns=2, SRFs, 1.46, 0.58, 1.60, 0.58, 
Np=2 Ns=1, SRFp, 2.99, 1.15, 3.00, 1.14, 
Np=2 Ns=1, SRFs, 82.98, 14.56, 85.31, 14.81, 
